# Продвинутый Python, лекция 9

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Бузаев Федор, Дешеулин Олег

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

## HTTP и иже с ним

Теперь приступим к новым вещам, а именно - к вебом и взаимодействием с ним!

![](https://www.thebozho.com/wp-content/uploads/2017/04/Free-Open-Internet.jpg)

Давайте начнем просто с того, что же такое HTTP? В переводе это HyperText Transfer Protocol, или же протокол для передачи т.н. HyperText (гиепртекст - это текст с наличием ссылок для перехода на другие странички, изи)

Что в таком случае https, который мы сейчас везде видим? Это тот же HTTP, но еще и S (secure) - данные передаются не просто открыто, а через протокол шифрования [TLS](https://habr.com/ru/post/258285/) (не будет вдаваться в подробности, как оно работает, речь не про это)

По сути своей, это модель "клиент-сервер". Клиент дает запрос, сервер отвечает, еще запрос, еще ответ etc хоть до бесконечности. Выглядит максимально просто и понятно

К примеру, когда вы заходите на сайт, вы отправляете запрос по типу "покажи сайт". Сервер такой: "Держи". Потом тыкаете на ссылку, опять запрос etc



Что происходит при открытии сайта?

Например, вы заходите в [питон](https://python.org/). Что необходимо сделать для этого браузеру?

Браузер отправляет запрос на сервер, который расположен по адресу python.org. Удивительно, но это можно сделать даже без браузера (кто бы мог подумать)

In [ ]:
!brew install telnet
!telnet python.org 80 # утилита, которая осуществляет запрос как клиентская часть

Подсоединились, теперь нам надо отправлять запросы, чтобы получать какие-то ответы. Как выглядят запросы в HTTP?

На самом деле тут все достаточно просто. Любой запрос в HTTP выглядит следующим образом:

```
Метод URI HTTP/1.1\n
Host: hostname\n
\n

```

* Метод - собственно операция, которую хотим осуществить

* URI - путь до документа/страницы, которая нам нужна

* HTTP/1.1 - указываем версию HTTP (современная - 1.1)

* hostname - название хоста (так как удалённый сервер не обладает никакой информацией о том, какой именно адрес использовался для соединения, то его надо указывать)

На самом деле можно указывать также дополнительные параметры (например, какой у нас User-Agent (браузер) etc)

Базовый метод - это GET (название говорит само за себя). Ну давайте получим информацию (перейдем на Python, не зря же его изучаем)

In [ ]:
import telnetlib #библиотека для подключений, будет deprecated

print("connecting...")
tn = telnetlib.Telnet("www.python.org", '80')
msg = "GET / HTTP/1.1\nHost:www.python.org\n\n".encode('ascii') #необходимо, чтобы отправить без пробразования \n в перевод строки внутри строки
tn.write(msg)
print(tn.read_all().decode("ascii"))
print("response received")
tn.close()

connecting...
HTTP/1.1 301 Moved Permanently
Connection: close
Content-Length: 0
Server: Varnish
Retry-After: 0
Location: https://www.python.org/
Accept-Ranges: bytes
Date: Sun, 09 Oct 2022 20:45:58 GMT
Via: 1.1 varnish
X-Served-By: cache-iad-kjyo7100086-IAD
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1665348358.489964,VS0,VE0
Strict-Transport-Security: max-age=63072000; includeSubDomains


response received


Что мы всегда видим в качестве ответа? В первую очередь строку следующего вида:

```
HTTP/1.1 Код Пояснение
```

* HTTP/1.1 - версия HTTP

* Код - [код выполнения запроса](https://developer.mozilla.org/ru/docs/Web/HTTP/Status)

Вот наши любимые:

1. 200 - все ок

2. 4** - вы лохb (сюда относится и 403, 404 - запрещено/не найдено)

3. 5** - мы лохи

* Пояснение - название кода ответа (типа OK, Not Found etc)


Дальше идет тело ответа: что собственно, мы запросили вообще (но никакого response нет, ибо 301)

Какие есть основные методы для HTTP?

* GET - получи инфу

* POST - запость инфу

* PUT - обнови инфу

* DELETE - удали инфу

Если вы решили, что можно взять и тогда удалить Python, то вот нет. Единственное, что обязан обрабатывать веб-сервер - это GET. Остального может не быть (и тогда сервер вас просто не поймет). А еще могут быть всякие другие методы

Понятное дело, что для этого должна быть какая-нибудь удобная библиотека (а не тупо telnet, в который еще как-то странно надо передавать данные). Для этого есть requests!

## Requests

[Requests](https://requests.readthedocs.io/en/latest/api/) - это отличная библиотека для работы с подключениями к сайту. Давайте разбираться на примерах:

In [ ]:
import requests

requests.get('https://api.github.com') #аналогия get, получаем ответ

<Response [200]>

Какие атрибуты есть у get?

In [ ]:
r = requests.get('https://api.github.com')
print(r.status_code, r.reason) # статус ответа (200 - все ОК)
print('-' * 30)
for i in r.headers.items(): # информация
    print(i[0], ':', i[1])
print('-' * 30)
print(r.encoding)
print('-' * 30)
print(r.text) # что вывели (тело) (в чистом виде можно с помощью .content, text сразу декодирует)
print('-' * 30)
print(r.url) # где находимся
print('-' * 30)
print(r.json) # и можно сразу в виде json

200 OK
------------------------------
Server : GitHub.com
Date : Sun, 09 Oct 2022 22:01:46 GMT
Content-Type : application/json; charset=utf-8
Cache-Control : public, max-age=60, s-maxage=60
Vary : Accept, Accept-Encoding, Accept, X-Requested-With
ETag : W/"4f825cc84e1c733059d46e76e6df9db557ae5254f9625dfe8e1b09499c449438"
X-GitHub-Media-Type : github.v3; format=json
Access-Control-Expose-Headers : ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset
Access-Control-Allow-Origin : *
Strict-Transport-Security : max-age=31536000; includeSubdomains; preload
X-Frame-Options : deny
X-Content-Type-Options : nosniff
X-XSS-Protection : 0
Referrer-Policy : origin-when-cross-origin, strict-origin-when-cross-origin
Content-Security-Policy : default-src 'none'
Content-Encod

Иногда мы хотим делать get не просто по сайту, а еще передавать какие-нибудь параметры. Давайте попробуем:

In [ ]:
response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'Deep_Python'}, #делаем поиск с помощью параметра q https://docs.github.com/en/rest/search
)

# Анализ некоторых атрибутов местонахождения запросов
json_response = response.json()
repository = json_response['items'][0]
print(f'Repository name: {repository["name"]}')
print(f'Repository description: {repository["description"]}')

Repository name: deep-learning-with-python-notebooks
Repository description: Jupyter notebooks for the code samples of the book "Deep Learning with Python"


In [ ]:
for i in range(len(json_response['items'])):
    if json_response['items'][i]['name'] == 'Deep_Python':
        print(i)

21


In [ ]:
json_response['items'][21]

Какие есть функции?

Есть самый основные для HTTP-протокола:

In [ ]:
requests.post('https://httpbin.org/post', data={'key':'value'}) #запостить инфу
requests.put('https://httpbin.org/put', data={'key':'value'}) #полная замена инфы (придется указать все, что даже не меняется)
requests.delete('https://httpbin.org/delete') #удалить инфу
requests.head('https://httpbin.org/get') #получить информацию без тела
requests.patch('https://httpbin.org/patch', data={'key':'value'}) #замена инфы, но без указания того, что не надо заменять

<Response [504]>

Иногда для того, чтобы получить доступ, надо сделать авторизацию (аутентификацию), для этого внутри requests есть

In [ ]:
from requests.auth import HTTPBasicAuth #самый простой способ аутентификации, почти везде принимается
from getpass import getpass #библиотека из 2 функций: запрос пароля и запрос пользователя

requests.get(
     'https://api.github.com/user',
     auth=HTTPBasicAuth('user', getpass())
)

··········


<Response [401]>

Давайте себе представим ситуацию: сервер подвис! Что же делать, наш код будет сидеть и ждать, а такого мы не хотим. Внутри requests можно задать время ожидания, который мы используем

In [ ]:
from requests.exceptions import Timeout

try:
    response = requests.get('https://api.github.com', timeout=1) #устанавливаем максимальный timeout, если не случилось, то он бросает Timeout
except Timeout:
    print('The request timed out')
else:
    print('The request did not time out')

The request did not time out


И последнее, что мы сегодня разберем, так это Session. Что это такое?

Сессия - это возможность сделать несколько обращений, при этом сохранив общие параметры, которые вам нужны (ну, например, если у вас вдруг везде одинаковый логин-пароль, можно его сохранить и использовать для нескольких вызовов)

In [ ]:
with requests.Session() as session:
    session.auth = ('username', getpass())
    session.headers.update({'one': 'true'})
    print(session.headers)
    response = session.get('https://api.github.com')

print(response.headers)
print(response.json())

··········
{'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'one': 'true'}
{'Server': 'GitHub.com', 'Date': 'Sun, 09 Oct 2022 23:57:32 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding, Accept, X-Requested-With', 'ETag': 'W/"4f825cc84e1c733059d46e76e6df9db557ae5254f9625dfe8e1b09499c449438"', 'X-GitHub-Media-Type': 'github.v3; format=json', 'X-RateLimit-Limit': '60', 'X-RateLimit-Remaining': '55', 'X-RateLimit-Reset': '1665362812', 'X-RateLimit-Used': '5', 'X-RateLimit-Resource': 'core', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, 

Наверное, вы слышали какую-нибудь фразу типа "REST API". Что это вообще значит?

Давайте разбираться по словам

## REST

REST (Representational State Transfer) - это, в сущности, некоторое соглашения по поводу того, как проектировать наши запросы и как с ними работать (то есть просто принципы, которым все следуют, чтобы было проще работать)

Что в себя включает REST?

1. Клиент-серверная архитектура (есть две части: клиент, который отправляет запросы и сервер, который принимает запросы и обрабатывает их. Эти части разделены). Пример не клиент-серверной архитектуры - блокчейн (клиент и сервер соединены, все самодостаточны)

2. Stateless (обращаясь к серверу, он нигде не запоминает сам запрос. А если храним, то только на стороне клиента)

3. Кэширование (можно использовать кэш, чтобы не обращаться дважды к другим источникам на основе уже ранее показанного)

4. Единообразие интерфейса (сервер возвращает не только ресурс, но и его связи с другими ресурсами и действия, которые можно с ним совершить)

5. Layered system (ни клиент, ни сервер не должны знать о том, как происходит цепочка вызовов дальше своих прямых соседей, то есть линейная система)

6. Code on demand (передаем сам код выполнения, а не только само действие)

Но что более интересно - каждый понимает REST по-своему, что несколько затрудняет нашу жизнь. Изложенное выше - это оригинальное трактование REST, но разные люди могут по-разному ее определять. Почему? Потому что это соглашения, а не железное правило. Но самое главное, что REST - это всегда про HTTP протоколы (и еще обычно говорят, что для REST всегда должны быть методы GET, DELETE, POST, PUT)

Теперь перейдем ко второму слову - API

## API

API (Application programming interface) - некоторое соглашение (или декларация) о том, какие функции есть, как к ним обращаться и как работать. По сути - документация

В качестве примера: https://requests.readthedocs.io/en/latest/api/ - вот это API

Тыкаем вызов функции - вызывается API, допустим pandas. А pandas еще внутри вызывает API numpy (они очень связаны). А если вы вызываете в seaborn отрисовку графика через данные в pandas, то вы понимаете, что происходит

И в совокупности REST API - это обмен данными между API по REST принципу. Все просто! Зачем так делать? Для простоты взаимодействия (все следуют общим правилам - есть общая система, где не надо вникать - успех!)

Зачем нам это все знать? Ну вот допустим, что вы решили сделать сайт, для него же надо сделать API, чтобы клиент мог обращаться, тыкаться и работать с ним. Вот для этого

На семинаре мы поговорим про Telegram API (и ботика напишем простого), а дальше будет еще лучше)

## Docker

### Но... зачем?

Теперь мы перешли к самой сложной части - про докер.

Что такое докер? Представьте себе ситуацию: у вас есть приложение, которое вы хотите запустить и показать всему миру, какие вы классные. Что делать?

Можно, конечно, создать виртуалку, на нее накинуть приложение и там все делать. Но в какой-то момент вам придется класть туда несколько приложений и дальше начинается катавасия с загруженностью процесса. В какой-то момент одно приложение может стрельнуть (например, если у вас отдельное приложение для новостей, куда зайдет сразу дофига людей), сожрет все процессы и остальные части будут страдать. А еще же люди заходят с разными ОС, и надо конфигурации разные иметь, чтобы у всех все работало. Вот для этих вещей и есть контейнеры

Что такое контейнер? Это способ стандартизации развертки приложения и отделения его от общей инфраструктуры. Экземпляр приложения запускается в изолированной среде, не влияющей на основную операционную систему


![](https://d1.awsstatic.com/Developer%20Marketing/containers/monolith_2-VM-vs-Containers.78f841efba175556d82f64d1779eb8b725de398d.png)

Контейнер позволяют:

1. упаковать в единый образ приложение и все его зависимости: библиотеки, системные утилиты и файлы настройки. Это упрощает перенос приложения на другую инфраструктуру

2. приложения работают только внутри контейнеров и не имеют доступа к основной операционной системе. Это повышает безопасность приложений:они не смогут случайно или умышленно навредить основной системе. Если приложение в контейнере завершится с ошибкой или зависнет, это никак не затронет основную ОС (изоляция ресурсов)

3. избавляет от зависимости ОС: достаточно добавить необходимую конфигурацию в контейнер вместо процесса эмулирования одной ОС на другой (что трудозатратно)

4. За счет оптимизации контейнеров получаем также меньшую загрузку

### Ну что же, начнем

Первое, что надо сделать - это загрузить образ из Docker Hub. Образ (Image) - это схема нашего приложения, основа контейнера, с помощью которого его можно запустить. Все возможные образы хранятся [здесь](https://hub.docker.com/search?q=&type=image) (можно будет добавить сюда же и свои образы, которые вам необходимы)

Загрузим самый простой образ - busybox (дальше будет работа в терминале, а не в Колабе)

In [ ]:
docker pull busybox # загрузи образ busybox, если Permission denied, то запустите с sudo
docker images # посмотреть на все загруженные образы
docker rmi busybox # удалить образ

Отлично, загрузили, давайте запускать!

In [ ]:
docker run busybox # ничего не случилось, потому что мы ничего и не задали
docker run busybox echo "Hello" # о, что-то выдал
docker ps # посмотреть запущенные контейнеры (их пока нет, потому что прошлые кончились)
docker ps -a # посмотреть все контейнеры
docker run -it busybox sh #запустить на больше, чем 1 команду (-it - флаг интерактива)

Все запушенные и кончившиеся контейнеры занимают место, поэтому их еще стоит удалять (как минимум те, которые завершили свое действие)

In [ ]:
docker rm <ids> # удалить все ненужные контейнеры
docker rm $(docker ps -a -q -f status=exited) # чтобы не копировать все id
# -a все контейнеры, -q  - вывести только ID, -f - фильтр (на статус закончившися exited)

Обратите внимание сверху на значок Docker (MacOS). Видим, что Docker работает. Что это значит? Что у нас запушен процесс, с помощью которого это все вообще происходит (так называемый Docker Daemon)

### Запустим сайт из под капота

Окей, повеселились с каким-то образом. Как приложения-то запускать? Давайте попробуем запустить какой-нибудь более интересный образ

In [ ]:
docker pull prakhar1989/static-site
docker run prakhar1989/static-site #ничего не происходит, видим просто что is running
docker stop $(docker ps -a -q -f status=running) #выключаем все активные контейнеры
docker run -d -P --name static-site prakhar1989/static-site #давайте сделаем так:
# -d - открепляем наш терминал от контейнера(не будет прикреплен, можем продолжить работу в терминале)
# -P - сделаем порты открытыми и публичными, чтобы подключиться
# --name переименуем для удоства в static-site
docker port static-site #смотрим на порты, открываем на localhost:<порт>

Ура, запустили простенький сайт! А если поменять порт? Можно

In [ ]:
docker run -d -P -p 8888:80 prakhar1989/static-site --name static-site prakhar1989/static-site #теперь он на порту 8888
# окей, остановим все и удалим

### Хотим быть крутыми со своим образом!

Теперь хотим создать свой собственный образ и сделать еще интереснее. Образы делятся на 2 типа:

* Официальные образы - поддерживаются командой docker, скачать можно по названию (например, docker pull python)

* Неофициальные образы - образ, созданный пользователем, чаще всего выглядит как user/name

Скачаем простенький сайт, сделанный на Flask (о нем мы будет говорить в других семинарах) и запустим его локально

In [ ]:
git clone https://github.com/prakhar1989/docker-curriculum.git #скачаем для простого приложения на Flask (о нем мы будет говорить в других семинарах)
cd docker-curriculum/flask-app; pip install -r requirements.txt #установим все, что нужно для сайта
vim app.py #для того, чтобы у меня запустилась, так как порт 5000 у меня занят, я поменял в коде на 8888
python app.py

Получилось! Теперь создадим образ с этим приложением. Что нужно?

Так как сайт написан на Python, то надо скачать базовый образ python

In [ ]:
docker pull python:3-onbuild #Какой onbuild, це шо? Это надстройка при запуске возьмет на requirements.txt и установит за нас все, что нужно
# То есть такой помощник при запуске

Теперь надо как-то соединить приложение и образ. Это делается с помощью DockerFile - текстовый документ для автоматизации сборки (код для DockerFile почти идентичен аналогам bash)

Создаем там же, где и наше приложение файл Dockerfile и прописываем:

In [ ]:
vim Dockerfile

In [ ]:
FROM python:3-onbuild #указываем образ, который надо использовать

#благодаря тому, что у нас onbuild, нам не надо копировать файлы и устанавливать зависимости здесь

EXPOSE 8888 #говорим, на какой порт это все отправлять

CMD ["python", "./app.py"] # команды для запуска (то есть что надо сделать)

Осталось собрать образ через docker build:

In [ ]:
docker build -t palladain7/catgif . #здесь надо зарегаться на Docker Hub (это быстро) и в качестве user ввести свой ник
docker run -p 8888:8888 palladain/catgif #собрали-запустили
docker images #проверяем образ

Ура, мы собрали образ докера! Теперь осталось его загрузить на Docker Hub, чтобы его могли увидеть все и использовать)

In [ ]:
docker login #вначале надо авторизоваться
docker push palladain7/catgif #пушим
https://hub.docker.com/r/palladain7/catgif/ - проверяем, успех

# Животное дня

![](https://cherepah.ru/wp-content/uploads/f/e/d/fed867b6875cf5a53f93f381c0aba25d.jpeg)

Это ленивец!

![](https://farm4.staticflickr.com/3194/2733424567_d254e0b359_b.jpg)

И это тоже ленивец, но они разные! В чем прикол?

Первый - это трехпалый ленивец, второй - двупалый ленивец, отличаются числом когтей (логично)

Но в чем главный прикол? Так в том, что они вообще не родственники! Они проихошли от совсем разных видов, но эволюция привела их к тому, чтобы быть ленивцами (прям как с крабами).

Ленивцы ленивые, буквально, они спят по 20 часов в сутки, а остальное время едят. За счет того, что они питаются листьями, то у них почти полностью атрофированная мускулатура (на дереве они держатся чисто за счет когтей), плохо видят. Но при этом достаточно неплохо слышат и плавают! (а ходят очень нелепо)

А еще у них 8микамерный желудок (занимает примерно 40% всего тела), что позволяет выжать максимум энергии из листьев. Они живут на деревьях, но при этом не умеют ходить в туалет с воздуха, поэтому раз в неделю им приходится спускаться (вот тогда их обычно и ловят)

Когда конкистадоры прибыли в Латинскую Америку, они пробовали использовать ленивцев в качестве пищи. Но когда они их подстреливали, ленивцы продолжали висеть высоко в деревьях, поэтому на это забили